In [3]:
import pandas as pd

train = pd.read_csv("Training.csv")
test = pd.read_csv("Testing.csv")
meds = pd.read_csv("cleaned_medicines.csv")

print("Training data:", train.shape)
print("Testing data:", test.shape)
print("Medicines data:", meds.shape)

display(train.head())
display(meds.head())


Training data: (4920, 134)
Testing data: (42, 133)
Medicines data: (23939, 5)


itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          1                     1                    0          0   
2        1          0                     1                    0          0   
3        1          1                     0                    0          0   
4        1          1                     1                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  scurring  \
0       0           0             0        0                 0  ...         0   
1       0           0             0        0                 0  ...         0   
2       0           0             0        0                 0  ...         0   
3       0           0             0        0                 0  ...         0   
4       0           0             0        0                 0  ...         0   

   skin_peeling  silver_like_dusting  small_dents_in_nails  \
0             0                    0                     0   
1             0                    0                     0   
2             0                    0                     0   
3             0                    0                     0   
4             0                    0                     0   

   inflammatory_nails  blister  red_sore_around_nose  yellow_crust_ooze  \
0                   0        0                     0                  0   
1                   0        0                     0                  0   
2                   0        0                     0                  0   
3                   0        0                     0                  0   
4                   0        0                     0                  0   

          prognosis  Unnamed: 133  
0  Fungal infection           NaN  
1  Fungal infection           NaN  
2  Fungal infection           NaN  
3  Fungal infection           NaN  
4  Fungal infection           NaN  

[5 rows x 134 columns]

disease_name                      med_name  \
0     ADHD (7)       Atrest 25mg Tablet 10'S   
1     ADHD (7)          Capnea Injection 1ml   
2     ADHD (7)      Capnea Oral Solution 1ml   
3     ADHD (7)  Cognistar 30mg Injection 1'S   
4     ADHD (7)  Cognistar 60mg Injection 1's   

                                     generic_name  \
0               Generic Name  Tetrabenazine 25 mg   
1                    Generic Name  Caffeine 20 mg   
2                    Generic Name  Caffeine 20 mg   
3  Generic Name  Cerebroprotein Hydrolysate 30 mg   
4  Generic Name  Cerebroprotein Hydrolysate 60 mg   

                                            img_urls  \
0  https://www.netmeds.com/images/product-v1/600x...   
1  https://www.netmeds.com/images/product-v1/600x...   
2  https://www.netmeds.com/images/product-v1/600x...   
3  https://www.netmeds.com/images/product-v1/600x...   
4  https://www.netmeds.com/images/product-v1/600x...   

                                             med_url  
0  https://www.netmeds.com/prescriptions/atrest-2...  
1  https://www.netmeds.com/prescriptions/capnea-i...  
2  https://www.netmeds.com/prescriptions/capnea-s...  
3  https://www.netmeds.com/prescriptions/cognista...  
4  https://www.netmeds.com/prescriptions/cognista...

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib, json

# Clean datasets: remove junk columns
train = train.loc[:, ~train.columns.str.contains("^Unnamed")]
test = test.loc[:, ~test.columns.str.contains("^Unnamed")]

# Features = all symptom columns (everything except prognosis)
SYMPTOM_COLS = [c for c in train.columns if c != "prognosis"]
LABEL_COL = "prognosis"

print("🩺 Symptom columns detected:", len(SYMPTOM_COLS))

# Define X and y
X_train = train[SYMPTOM_COLS].values
X_test = test[SYMPTOM_COLS].values

y_train_raw = train[LABEL_COL]
y_test_raw = test[LABEL_COL]

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train_raw)
y_test = le.transform(y_test_raw)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape, "Unique classes:", len(le.classes_))

# Train model
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("✅ Accuracy:", acc)

# Full classification report
report = classification_report(
    y_test,
    y_pred,
    target_names=le.classes_,
    output_dict=True
)

# Convert to DataFrame and save
df_report = pd.DataFrame(report).transpose()
df_report.to_csv("classification_report.csv", index=True)
df_report.to_excel("classification_report.xlsx", index=True)

print("📊 Classification report saved as 'classification_report.csv' and 'classification_report.xlsx'")

# Save artifacts for later use in Flask
joblib.dump(clf, "model.joblib")
joblib.dump(le, "label_encoder.joblib")
with open("symptom_list.json", "w") as f:
    json.dump(SYMPTOM_COLS, f)

print("💾 Model + artifacts saved: model.joblib, label_encoder.joblib, symptom_list.json")


🩺 Symptom columns detected: 132
X_train shape: (4920, 132)
X_test shape: (42, 132)
y_train shape: (4920,) Unique classes: 41


d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is g

✅ Accuracy: 0.9761904761904762
📊 Classification report saved as 'classification_report.csv' and 'classification_report.xlsx'
💾 Model + artifacts saved: model.joblib, label_encoder.joblib, symptom_list.json


In [5]:
# Save model + encoder
joblib.dump(clf, "model.joblib")
joblib.dump(le, "label_encoder.joblib")

# Save symptom list (all feature columns)
with open("symptom_list.json", "w") as f:
    json.dump(SYMPTOM_COLS, f)

# Build disease → medicines map
disease_col = "disease" if "disease" in meds.columns else meds.columns[0]
medicine_col = "med_name" if "med_name" in meds.columns else meds.columns[1]

medications = meds.groupby(disease_col)[medicine_col].apply(list).to_dict()

with open("medications.json", "w") as f:
    json.dump(medications, f)

print("✅ Saved: model.joblib, label_encoder.joblib, symptom_list.json, medications.json")



✅ Saved: model.joblib, label_encoder.joblib, symptom_list.json, medications.json


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import joblib, json, numpy as np
import re
from typing import List, Set

app = Flask(__name__)
CORS(app)  # allow localhost frontend

# -----------------------------
# Load artifacts
# -----------------------------
model = joblib.load("model.joblib")
label_encoder = joblib.load("label_encoder.joblib")

with open("symptom_list.json", "r") as f:
    symptom_list: List[str] = json.load(f)

with open("medications.json", "r") as f:
    medications = json.load(f)

# -----------------------------
# Normalization & matching
# -----------------------------
def normalize_text(text: str) -> str:
    """
    Lowercase, remove non-alphanumerics except spaces/underscores,
    collapse spaces, and replace spaces with underscores.
    """
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s_]", " ", text)  # keep underscores
    text = re.sub(r"\s+", " ", text).strip()
    text = text.replace(" ", "_")
    return text

# Normalize stored vocab once
symptom_list = [normalize_text(s) for s in symptom_list]
symptom_index = {s: i for i, s in enumerate(symptom_list)}

def extract_symptoms(payload_symptoms) -> List[str]:
    """
    Accepts either:
      - string: free text like "fever, headache and cough"
      - list:   ["fever", "headache"]

    Returns a list of matched vocab symptoms (normalized), deduplicated.
    """
    matched: Set[str] = set()

    if isinstance(payload_symptoms, list):
        # Treat as explicit list of tokens; normalize each and keep if in vocab
        for tok in payload_symptoms:
            tok_n = normalize_text(str(tok))
            if tok_n in symptom_index:
                matched.add(tok_n)
        return sorted(matched)

    # Otherwise, treat as free text (string)
    raw = str(payload_symptoms)
    # Make commas into spaces to avoid empty tokens
    raw = raw.replace(",", " ")
    full_norm = normalize_text(raw)

    # Strategy: scan the FULL normalized text for *any* vocab symptom as substring.
    # This supports multi-word symptoms like "high_fever", "chest_pain", etc.
    for s in symptom_list:
        if s and s in full_norm:
            matched.add(s)

    # Fallback: also split into tokens and keep exact matches if any
    tokens = [normalize_text(t) for t in raw.split()]
    for t in tokens:
        if t in symptom_index:
            matched.add(t)

    return sorted(matched)

def symptoms_to_vector(symptoms: List[str]) -> np.ndarray:
    x = np.zeros(len(symptom_list), dtype=int)
    for s in symptoms:
        idx = symptom_index.get(s)
        if idx is not None:
            x[idx] = 1
    return x.reshape(1, -1)

# -----------------------------
# Routes
# -----------------------------
@app.route("/", methods=["GET"])
def health():
    return jsonify({"message": "✅ Flask API is running. POST /predict with {'symptoms': <string|list>}"}), 200

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json(silent=True) or {}
        print("📥 Raw input:", data)

        payload_symptoms = data.get("symptoms")
        if payload_symptoms is None or (isinstance(payload_symptoms, str) and not payload_symptoms.strip()):
            return jsonify({"error": "No symptoms provided."}), 400

        matched = extract_symptoms(payload_symptoms)
        print("🧩 Matched symptoms:", matched)

        if not matched:
            return jsonify({
                "matched_symptoms": [],
                "predictions": [],
                "medications": [],
                "message": "No valid symptoms recognized. Please try again."
            }), 200

        X = symptoms_to_vector(matched)

        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(X)[0]
            top_idx = np.argsort(proba)[::-1][:3]
            preds = [
                {"disease": label_encoder.inverse_transform([i])[0], "confidence": float(proba[i])}
                for i in top_idx
            ]
        else:
            pred = model.predict(X)[0]
            preds = [{"disease": label_encoder.inverse_transform([pred])[0], "confidence": None}]

        top1 = preds[0]["disease"]
        meds = medications.get(top1, ["Consult a doctor"])

        print("🎯 Top disease:", top1, " meds:", meds[:5])

        return jsonify({
            "matched_symptoms": matched,
            "predictions": preds,
            "disease": top1,
            "confidence": preds[0]["confidence"],
            "medications": meds,
            "disclaimer": "This is not a medical diagnosis. Seek professional care for severe or persistent symptoms."
        }), 200

    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    # Use port 5000 to match your frontend fetch("http://127.0.0.1:5000/predict", ...)
    app.run(host="127.0.0.1", port=5001, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


In [8]:
# main_app.ipynb

# --- Cell 1: Imports & Config ---
import os
import pandas as pd
from flask import Flask, request, jsonify
import openai
from dotenv import load_dotenv
from flask_cors import CORS
import joblib, json, numpy as np
import re
from typing import List, Set
from werkzeug.serving import run_simple

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize Flask app
app = Flask(__name__)
CORS(app) 

# --- Cell 2: Load Dataset ---
# Load your dataset
df = pd.read_excel("symptom_list.csv.xlsx", sheet_name="Sheet1")

# Utility function to split symptoms into list
def split_symptoms(sym_str):
    return [s.strip().lower() for s in str(sym_str).split(",") if s.strip()]

# --- Cell 3: Predict Route ---
@app.route("/")
def index():
    return "Server OK (Flask running on 127.0.0.1:5005)"

@app.route("/predict", methods=["POST"])
def predict():
    """
    Step 1:
    - Take user symptoms (speech-to-text transcript).
    - Match with 'Typical Symptoms'.
    - Return possible diseases + a clarifying question.
    """
    data = request.get_json()
    user_text = " ".join(data.get("symptoms", [])).lower()

    possible = []
    for _, row in df.iterrows():
        typical = split_symptoms(row["Typical Symptoms"])
        if any(sym in user_text for sym in typical):
            possible.append({
                "disease": row["Disease"],
                "confirm_symptoms": split_symptoms(row["Confirm Symptom"]),
                "unique_symptom": row["Unique Symptom"]
            })

    if not possible:
        return jsonify({"stage": "no_match", "message": "No disease match found."})

    # Build confirm symptom list
    confirm_list = [", ".join(p["confirm_symptoms"]) for p in possible]

    # Ask AI to phrase the clarifying question
    try:
        question = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a friendly medical assistant."},
                {"role": "user", "content": f"User reported: {user_text}. Possible diseases: {[p['disease'] for p in possible]}. Confirm symptoms to ask about: {confirm_list}. Ask the user politely about these symptoms in one short question."}
            ]
        )
        ai_question = question["choices"][0]["message"]["content"]
    except Exception as e:
        ai_question = f"Do you have any of these symptoms? {confirm_list}"

    return jsonify({
        "stage": "ask_confirmation",
        "possible_diseases": possible,
        "question": ai_question
    })

# --- Cell 4: Confirm Route ---
@app.route("/confirm", methods=["POST"])
def confirm():
    """
    Step 2:
    - Take user reply + shortlisted diseases.
    - Match with 'Confirm Symptom'.
    - Return final most likely disease.
    """
    data = request.get_json()
    user_reply = data.get("reply", "").lower()
    possible = data.get("possible_diseases", [])

    best_match = None
    best_count = 0
    for p in possible:
        count = sum(sym.lower() in user_reply for sym in p["confirm_symptoms"])
        if count > best_count:
            best_count = count
            best_match = p["disease"]

    if best_match:
        return jsonify({
            "stage": "final_result",
            "disease": best_match,
            "confidence": f"{best_count} confirm symptoms matched"
        })
    else:
        return jsonify({
            "stage": "final_result",
            "disease": "Unclear",
            "confidence": "No confirm symptoms matched"
        })

# --- Cell 5: Run Flask ---
if __name__ == "__main__":
    run_simple("127.0.0.1", 5005, app, use_reloader=False, use_debugger=True)


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [20/Sep/2025 23:27:50] "OPTIONS /predict HTTP/1.1" 200 -
[2025-09-20 23:27:50,687] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "d:\New folder (3)\Lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Confirm Symptom'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\New folder (3)\Lib\site-packages\flask\app.py", line 1511, in w

In [5]:
# put after df = ... load call
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
app = Flask(__name__)
CORS(app) 
df = pd.read_excel("symptom_list.csv.xlsx", sheet_name="Sheet1")
app.logger.info("Server started. Symptom file exists: %s", os.path.exists("symptom_list.csv") or os.path.exists("symptom_list.csv.xlsx"))
app.logger.info("DF shape: %s, columns: %s", getattr(df, "shape", "NA"), getattr(df, "columns", "NA").tolist())

# debug endpoint
@app.route("/_debug_symptoms", methods=["GET"])
def debug_symptoms():
    return jsonify({
        "shape": list(df.shape),
        "columns": df.columns.tolist(),
        "sample": df.iloc[:3].to_dict(orient="records")
    })


In [2]:
# main_app_safe.py  (notebook-friendly)
import os, re, json, traceback
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import openai
from dotenv import load_dotenv

# ---------- CONFIG ----------
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY", "")  # optional

DATA_PATHS = [
    "symptom_list.csv",
    "symptom_list.xlsx",
    "symptom_list.csv.xlsx",
    "./data/symptom_list.csv",
    "./data/symptom_list.xlsx"
]

# ---------- APP ----------
app = Flask(__name__)
CORS(app)

# ---------- UTIL ----------
def normalize_text(s):
    return re.sub(r"[^a-z0-9\s]", " ", str(s or "").lower())

def token_set(s):
    return set(t for t in normalize_text(s).split() if t)

def split_symptoms(sym_str):
    return [s.strip() for s in str(sym_str).split(",") if s.strip()]

# ---------- LOAD DATA (normalize colnames, strict validation) ----------
def load_symptom_df():
    found = None
    for p in DATA_PATHS:
        if os.path.exists(p):
            found = p
            break
    if not found:
        raise FileNotFoundError(
            f"No symptom file found. Searched: {DATA_PATHS}. Put symptom_list.csv or symptom_list.xlsx in the app folder or change DATA_PATHS."
        )

    # read file (CSV or Excel)
    try:
        if found.lower().endswith(".csv"):
            df = pd.read_csv(found)
        else:
            try:
                df = pd.read_excel(found, sheet_name="Sheet1")
            except Exception:
                df = pd.read_excel(found)
    except Exception as e:
        raise RuntimeError(f"Failed to read symptom file {found}: {e}")

    # ---- normalize column names: strip & collapse spaces ----
    new_cols = []
    for c in df.columns:
        nc = str(c).strip()
        nc = " ".join(nc.split())
        new_cols.append(nc)
    df.columns = new_cols

    # ---- optional mapping for common alternate names ----
    col_map = {
        "Confirm Symptom ": "Confirm Symptom",
        "confirm symptom": "Confirm Symptom",
        "typical symptoms": "Typical Symptoms",
        "unique symptom": "Unique Symptom",
        "disease name": "Disease"
    }
    # normalize mapping keys
    normalized_map = {}
    for k, v in col_map.items():
        nk = " ".join(str(k).strip().split())
        normalized_map[nk.lower()] = v

    # build rename lookup using lowered keys
    rename_lookup = {}
    for c in df.columns:
        key = " ".join(str(c).strip().split()).lower()
        if key in normalized_map:
            rename_lookup[c] = normalized_map[key]
    if rename_lookup:
        df.rename(columns=rename_lookup, inplace=True)

    # Basic validation
    required = {"Disease", "Typical Symptoms", "Confirm Symptom", "Unique Symptom"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Dataset missing required columns: {missing}. Columns found: {list(df.columns)}")

    return df, found

# Attempt load and surface clear errors (no sys.exit)
try:
    df, data_file_used = load_symptom_df()
    app.logger.info("Loaded symptom data from %s — shape %s", data_file_used, df.shape)
    app.logger.info("Columns found: %s", df.columns.tolist())
except Exception as e:
    # Print full traceback and a helpful message, but DON'T exit the kernel.
    tb = traceback.format_exc()
    print("ERROR loading symptom dataset:\n", tb)
    # To help debugging, expose a lightweight debug endpoint that shows the error
    data_file_used = None
    df = None

    @app.route("/")
    def index_error():
        return "Server started but dataset failed to load. Check notebook output for the traceback."

    @app.route("/_debug_load_error", methods=["GET"])
    def _debug_load_error():
        return jsonify({
            "error": str(e),
            "traceback": tb,
            "searched_paths": DATA_PATHS
        })

# ---------- If df is loaded, normal endpoints; else endpoints still run but return helpful errors ----------
@app.route("/_debug_symptoms", methods=["GET"])
def debug_symptoms():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error or notebook output."}), 500
    try:
        return jsonify({
            "file": data_file_used,
            "shape": list(df.shape),
            "columns": df.columns.tolist(),
            "sample_first_rows": df.head(5).to_dict(orient="records")
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/predict", methods=["POST"])
def predict():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error or notebook output."}), 500

    data = request.get_json(force=True, silent=False)
    app.logger.info("Predict request JSON: %s", data)
    tokens = data.get("symptoms", [])

    if isinstance(tokens, str):
        tokens = re.findall(r"\w+", tokens.lower())
    if not isinstance(tokens, (list, tuple)):
        return jsonify({"error": "Invalid payload: 'symptoms' must be list or string"}), 400

    user_text = " ".join(str(t) for t in tokens).lower()
    user_tokens = token_set(user_text)

    possible = []
    for _, row in df.iterrows():
        typical_phrases = split_symptoms(row["Typical Symptoms"])
        phrase_match = any(normalize_text(ph) in normalize_text(user_text) for ph in typical_phrases if ph)
        token_overlap = sum(len(user_tokens & token_set(ph)) for ph in typical_phrases)
        if phrase_match or token_overlap >= 1:
            possible.append({
                "disease": row["Disease"],
                "confirm_symptoms": split_symptoms(row["Confirm Symptom"]),
                "unique_symptom": row.get("Unique Symptom", "")
            })

    app.logger.info("Possible matches found: %s", [p["disease"] for p in possible])
    if not possible:
        return jsonify({"stage": "no_match", "message": "No disease match found."})

    confirm_list = [", ".join(p["confirm_symptoms"]) for p in possible]

    try:
        if openai.api_key:
            question = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a friendly medical assistant."},
                    {"role": "user", "content": f"User reported: {user_text}. Possible diseases: {[p['disease'] for p in possible]}. Confirm symptoms to ask about: {confirm_list}. Ask the user politely about these symptoms in one short question."}
                ],
                max_tokens=80
            )
            ai_question = question["choices"][0]["message"]["content"]
        else:
            raise RuntimeError("OpenAI key not present - using fallback.")
    except Exception as e:
        app.logger.info("OpenAI fallback: %s", e)
        ai_question = f"Please tell me if you have any of these symptoms: {confirm_list}"

    return jsonify({
        "stage": "ask_confirmation",
        "possible_diseases": possible,
        "question": ai_question
    })

@app.route("/confirm", methods=["POST"])
def confirm():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error or notebook output."}), 500

    data = request.get_json(force=True, silent=False)
    user_reply = data.get("reply", "")
    possible = data.get("possible_diseases", [])

    if not isinstance(possible, list):
        return jsonify({"error": "Invalid payload: 'possible_diseases' must be a list"}), 400

    reply_text = str(user_reply).lower()
    best_match = None
    best_count = 0
    for p in possible:
        count = sum(1 for sym in p.get("confirm_symptoms", []) if sym.lower() in reply_text)
        if count > best_count:
            best_count = count
            best_match = p.get("disease")

    if best_match:
        return jsonify({"stage":"final_result","disease":best_match,"confidence":f"{best_count} confirm symptoms matched"})
    else:
        return jsonify({"stage":"final_result","disease":"Unclear","confidence":"No confirm symptoms matched"})

# ---------- RUN ----------
if __name__ == "__main__":
    # Use use_reloader=False to avoid child-process reloader in notebooks which causes confusing SystemExit messages
    app.run(host="127.0.0.1", port=5005, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2025 00:10:43] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 00:10:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 01:56:34] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 01:56:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 01:56:45] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 01:56:45] "POST /confirm HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:00:40] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:00:40] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:01:40] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:01:40] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:02:01] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:02:01] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:02:14] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 02:02:14] "

In [1]:
# main_app_safe.py  (notebook-friendly)
import os, re, json, traceback
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import openai
from dotenv import load_dotenv
from difflib import SequenceMatcher
import time
import threading

# ---------- CONFIG ----------
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY", "")  # optional

DATA_PATHS = [
    "symptom_list.csv",
    "symptom_list.xlsx",
    "symptom_list.csv.xlsx",
    "symptom_list_with_recommendations.xlsx",
    "./data/symptom_list.csv",
    "./data/symptom_list.xlsx"
]

# ---------- APP ----------
app = Flask(__name__)
CORS(app)

# ---------- UTIL ----------
def normalize_text(s):
    return re.sub(r"[^a-z0-9\s]", " ", str(s or "").lower())

def token_set(s):
    return set(t for t in normalize_text(s).split() if t)

def split_symptoms(sym_str):
    return [s.strip() for s in str(sym_str).split(",") if s.strip()]

# fuzzy similarity helper
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# ---------- emergency keywords ----------
EMERGENCY_KEYWORDS = [
    "chest pain", "breathless", "shortness of breath", "difficulty breathing",
    "bleeding", "vomit blood", "blood in stool", "severe", "collapse",
    "unconscious", "faint", "loss of consciousness", "severe pain", "dangerous",
    "difficulty breathing", "rapid breathing", "painful breathing"
]

def detect_emergency(text):
    if not text:
        return False
    t = str(text).lower()
    for kw in EMERGENCY_KEYWORDS:
        if kw in t:
            return True
    return False

# ---------- LOAD DATA (normalize colnames, strict validation) ----------
_lock = threading.Lock()
_cache = {"df": None, "loaded_at": None, "path": None}

def load_symptom_df():
    with _lock:
        if _cache["df"] is not None:
            return _cache["df"], _cache["path"]

        found = None
        for p in DATA_PATHS:
            if os.path.exists(p):
                found = p
                break
        if not found:
            raise FileNotFoundError(
                f"No symptom file found. Searched: {DATA_PATHS}. Put symptom_list.csv or symptom_list.xlsx in the app folder or change DATA_PATHS."
            )

        # read file (CSV or Excel)
        try:
            if found.lower().endswith(".csv"):
                df = pd.read_csv(found)
            else:
                try:
                    df = pd.read_excel(found, sheet_name="Sheet1")
                except Exception:
                    df = pd.read_excel(found)
        except Exception as e:
            raise RuntimeError(f"Failed to read symptom file {found}: {e}")

        # ---- normalize column names: strip & collapse spaces ----
        new_cols = []
        for c in df.columns:
            nc = str(c).strip()
            nc = " ".join(nc.split())
            new_cols.append(nc)
        df.columns = new_cols

        # ---- optional mapping for common alternate names ----
        col_map = {
            "Confirm Symptom ": "Confirm Symptom",
            "confirm symptom": "Confirm Symptom",
            "typical symptoms": "Typical Symptoms",
            "unique symptom": "Unique Symptom",
            "disease name": "Disease"
        }
        # normalize mapping keys
        normalized_map = {}
        for k, v in col_map.items():
            nk = " ".join(str(k).strip().split())
            normalized_map[nk.lower()] = v

        # build rename lookup using lowered keys
        rename_lookup = {}
        for c in df.columns:
            key = " ".join(str(c).strip().split()).lower()
            if key in normalized_map:
                rename_lookup[c] = normalized_map[key]
        if rename_lookup:
            df.rename(columns=rename_lookup, inplace=True)

        # Basic validation (these are still required for your existing predict/confirm flow)
        required = {"Disease", "Typical Symptoms", "Confirm Symptom", "Unique Symptom"}
        missing = required - set(df.columns)
        if missing:
            raise ValueError(f"Dataset missing required columns: {missing}. Columns found: {list(df.columns)}")

        # cache and return
        _cache["df"] = df
        _cache["loaded_at"] = time.time()
        _cache["path"] = found
        return df, found

# Attempt load and surface clear errors (no sys.exit)
try:
    df, data_file_used = load_symptom_df()
    app.logger.info("Loaded symptom data from %s — shape %s", data_file_used, df.shape)
    app.logger.info("Columns found: %s", df.columns.tolist())
except Exception as e:
    # Print full traceback and a helpful message, but DON'T exit the kernel.
    tb = traceback.format_exc()
    print("ERROR loading symptom dataset:\n", tb)
    # To help debugging, expose a lightweight debug endpoint that shows the error
    data_file_used = None
    df = None

    @app.route("/")
    def index_error():
        return "Server started but dataset failed to load. Check notebook output for the traceback."

    @app.route("/_debug_load_error", methods=["GET"])
    def _debug_load_error():
        return jsonify({
            "error": str(e),
            "traceback": tb,
            "searched_paths": DATA_PATHS
        })

# ---------- If df is loaded, normal endpoints; else endpoints still run but return helpful errors ----------
@app.route("/_debug_symptoms", methods=["GET"])
def debug_symptoms():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error or notebook output."}), 500
    try:
        return jsonify({
            "file": data_file_used,
            "shape": list(df.shape),
            "columns": df.columns.tolist(),
            "sample_first_rows": df.head(5).to_dict(orient="records")
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/predict", methods=["POST"])
def predict():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error or notebook output."}), 500

    data = request.get_json(force=True, silent=False)
    app.logger.info("Predict request JSON: %s", data)
    tokens = data.get("symptoms", [])

    if isinstance(tokens, str):
        tokens = re.findall(r"\w+", tokens.lower())
    if not isinstance(tokens, (list, tuple)):
        return jsonify({"error": "Invalid payload: 'symptoms' must be list or string"}), 400

    user_text = " ".join(str(t) for t in tokens).lower()
    user_tokens = token_set(user_text)

    possible = []
    for _, row in df.iterrows():
        typical_phrases = split_symptoms(row["Typical Symptoms"])
        phrase_match = any(normalize_text(ph) in normalize_text(user_text) for ph in typical_phrases if ph)
        token_overlap = sum(len(user_tokens & token_set(ph)) for ph in typical_phrases)
        if phrase_match or token_overlap >= 1:
            possible.append({
                "disease": row["Disease"],
                "confirm_symptoms": split_symptoms(row["Confirm Symptom"]),
                "unique_symptom": row.get("Unique Symptom", "")
            })

    app.logger.info("Possible matches found: %s", [p["disease"] for p in possible])
    if not possible:
        return jsonify({"stage": "no_match", "message": "No disease match found."})

    confirm_list = [", ".join(p["confirm_symptoms"]) for p in possible]

    try:
        if openai.api_key:
            question = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a friendly medical assistant."},
                    {"role": "user", "content": f"User reported: {user_text}. Possible diseases: {[p['disease'] for p in possible]}. Confirm symptoms to ask about: {confirm_list}. Ask the user politely about these symptoms in one short question."}
                ],
                max_tokens=80
            )
            ai_question = question["choices"][0]["message"]["content"]
        else:
            raise RuntimeError("OpenAI key not present - using fallback.")
    except Exception as e:
        app.logger.info("OpenAI fallback: %s", e)
        ai_question = f"Please tell me if you have any of these symptoms: {confirm_list}"

    return jsonify({
        "stage": "ask_confirmation",
        "possible_diseases": possible,
        "question": ai_question
    })

@app.route("/confirm", methods=["POST"])
def confirm():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error or notebook output."}), 500

    data = request.get_json(force=True, silent=False)
    user_reply = data.get("reply", "")
    possible = data.get("possible_diseases", [])

    if not isinstance(possible, list):
        return jsonify({"error": "Invalid payload: 'possible_diseases' must be a list"}), 400

    reply_text = str(user_reply).lower()
    best_match = None
    best_count = 0
    for p in possible:
        count = sum(1 for sym in p.get("confirm_symptoms", []) if sym.lower() in reply_text)
        if count > best_count:
            best_count = count
            best_match = p.get("disease")

    if best_match:
        return jsonify({"stage":"final_result","disease":best_match,"confidence":f"{best_count} confirm symptoms matched"})
    else:
        return jsonify({"stage":"final_result","disease":"Unclear","confidence":"No confirm symptoms matched"})

#
# ----------------------- NEW: /recommend endpoint -----------------------
#
# Uses the loaded dataframe to find a matching "Recommendations" entry.
# Returns:
#   {
#     ok: True,
#     matched_disease: "...",
#     advice_text: "full text for UI",
#     advice_html: "<div>...</div>",
#     advice_text_for_speech: "I'm sorry you're not feeling well ...",
#     severity: "...",
#     urgent: False,
#     row_index: 12,
#     source: "excel"
#   }
#
def find_recommendation_row(disease, tokens):
    """
    Returns (idx, row) or (None, None).
    Matching strategy:
      1) exact / normalized disease substring match
      2) token overlap with relevant columns (Typical Symptoms, Confirm Symptom, Recommendations)
      3) fuzzy similarity against disease-like columns
    """
    if df is None or df.empty:
        return None, None

    # detect recommendations column
    rec_col = None
    for c in df.columns:
        if "recommend" in c.lower():
            rec_col = c
            break

    # find disease-like columns
    disease_cols = [c for c in df.columns if 'disease' in c.lower() or 'name' in c.lower()]

    norm_disease = ""
    if disease:
        norm_disease = re.sub(r'[^a-z0-9 ]', ' ', str(disease).lower()).strip()

    # 1) exact substring disease match (preferred)
    if norm_disease:
        # search disease cols first
        for col in disease_cols:
            for idx, val in df[col].fillna("").iteritems():
                try:
                    if norm_disease and norm_disease in re.sub(r'[^a-z0-9 ]',' ', str(val).lower()):
                        return idx, df.loc[idx]
                except Exception:
                    continue
        # fallback: search whole row text
        for idx, row in df.iterrows():
            combined = " ".join([str(row.get(c,"")) for c in df.columns])
            if norm_disease and norm_disease in re.sub(r'[^a-z0-9 ]',' ', combined.lower()):
                return idx, row

    # 2) token overlap
    token_set_input = set(t.lower() for t in (tokens or []) if t)
    if token_set_input:
        best_idx = None
        best_score = 0
        for idx, row in df.iterrows():
            hay = ""
            # include columns likely to contain symptoms/recommendations
            for c in df.columns:
                if any(k in c.lower() for k in ['typical', 'confirm', 'symptom', 'recommend', 'unique', 'name', 'disease']):
                    hay += " " + str(row.get(c, ""))
            hay_norm = re.sub(r'[^a-z0-9 ]',' ', hay.lower())
            hay_words = set(hay_norm.split())
            score = len(token_set_input & hay_words)
            if score > best_score:
                best_score = score
                best_idx = idx
        if best_idx is not None and best_score > 0:
            return best_idx, df.loc[best_idx]

    # 3) fuzzy similarity against disease-like columns
    if disease_cols and norm_disease:
        best_idx = None
        best_sim = 0.0
        for col in disease_cols:
            for idx, val in df[col].fillna("").iteritems():
                sim = similarity(norm_disease, re.sub(r'[^a-z0-9 ]',' ', str(val).lower()))
                if sim > best_sim:
                    best_sim = sim
                    best_idx = idx
        if best_sim > 0.6:
            return best_idx, df.loc[best_idx]

    return None, None

def short_summary_for_tts(text, max_chars=220):
    if not text:
        return ""
    # take first sentence if short, else truncate gracefully
    parts = re.split(r'(?<=[.!?])\s+', text.strip())
    if parts and parts[0]:
        s = parts[0].strip()
        if len(s) <= max_chars:
            return s
    # fallback: truncate full text
    clean = re.sub(r'\s+', ' ', str(text).strip())
    if len(clean) <= max_chars:
        return clean
    truncated = clean[:max_chars].rsplit(' ', 1)[0]
    return truncated + "..."

@app.route("/recommend", methods=["POST"])
def recommend():
    """
    Expects JSON: { "disease": "Bronchitis ...", "tokens": ["fever","cough"] }
    Returns the recommendation record from the spreadsheet (or a fallback).
    """
    if df is None:
        return jsonify({"ok": False, "message": "Dataset not loaded. See /_debug_load_error."}), 500

    payload = request.get_json(silent=True) or {}
    disease = payload.get("disease", "") or ""
    tokens = payload.get("tokens", []) or []

    try:
        idx, row = find_recommendation_row(disease, tokens)
    except Exception as e:
        app.logger.exception("Error finding recommendation row: %s", e)
        idx, row = None, None

    # attempt to detect recommendations column
    rec_col = None
    for c in df.columns:
        if "recommend" in c.lower():
            rec_col = c
            break

    if row is None:
        # fallback generic advice
        generic_text = "I'm sorry you're not feeling well — rest, hydrate, and take paracetamol for fever if needed. If you have severe symptoms such as chest pain or difficulty breathing, seek emergency care."
        return jsonify({
            "ok": True,
            "matched_disease": None,
            "advice_text": generic_text,
            "advice_html": f"<div>{generic_text}</div>",
            "advice_text_for_speech": generic_text,
            "severity": "",
            "urgent": detect_emergency(generic_text),
            "row_index": None,
            "source": "fallback"
        })

    # Compose recommendation text
    rec_text = ""
    if rec_col:
        rec_text = str(row.get(rec_col, "") or "").strip()
    if not rec_text:
        # try to stitch together from other helpful columns
        parts = []
        for c in df.columns:
            if c.lower() not in (rec_col or "").lower():
                try:
                    v = str(row.get(c, "") or "").strip()
                except Exception:
                    v = ""
                if v:
                    parts.append(v)
        rec_text = " ".join(parts).strip()

    if not rec_text:
        rec_text = "No OTC suggestion; seek clinician. Rest and hydrate."

    # Prepare speech-friendly summary
    tts_short = short_summary_for_tts(rec_text, max_chars=220)
    empathetic_speech = f"I'm sorry you're not feeling well — {tts_short}. I can save your phone number for a doctor callback if you want."

    urgent = detect_emergency(rec_text)

    matched_name = None
    # try to get a matched disease name from any disease-like column
    for c in df.columns:
        if 'disease' in c.lower() or 'name' in c.lower():
            try:
                candidate = row.get(c)
                if candidate and str(candidate).strip():
                    matched_name = str(candidate)
                    break
            except Exception:
                continue
    if not matched_name:
        matched_name = disease or ""

    safe_html = f"<div><strong>Advice</strong><div style='margin-top:6px'>{rec_text}</div></div>"

    response = {
        "ok": True,
        "matched_disease": matched_name,
        "advice_text": rec_text,
        "advice_html": safe_html,
        "advice_text_for_speech": empathetic_speech,
        "severity": str(row.get("Severity")) if "Severity" in row.index else "",
        "urgent": bool(urgent),
        "row_index": int(idx) if idx is not None else None,
        "source": "excel"
    }
    return jsonify(response)

# ---------- RUN ----------
if __name__ == "__main__":
    # Use use_reloader=False to avoid child-process reloader in notebooks which causes confusing SystemExit messages
    app.run(host="127.0.0.1", port=5005, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2025 23:55:13] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 23:55:13] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 23:55:23] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 23:55:23] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 23:55:38] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2025 23:55:38] "POST /confirm HTTP/1.1" 200 -


In [2]:
# main_app_safe.py  (notebook-friendly)
import os, re, json, traceback
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import openai
from dotenv import load_dotenv
from difflib import SequenceMatcher
import time
import threading

# ---------- CONFIG ----------
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY", "")  # optional

DATA_PATHS = [
    "symptom_list.csv",
    "symptom_list.xlsx",
    "symptom_list.csv.xlsx",
    "symptom_list_with_recommendations.csv",
    "./data/symptom_list.csv",
    "./data/symptom_list.xlsx"
]

# ---------- APP ----------
app = Flask(__name__)
CORS(app)

# ---------- UTIL ----------
def normalize_text(s):
    return re.sub(r"[^a-z0-9\s]", " ", str(s or "").lower())

def token_set(s):
    return set(t for t in normalize_text(s).split() if t)

def split_symptoms(sym_str):
    return [s.strip() for s in str(sym_str).split(",") if s.strip()]

# fuzzy similarity helper
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# ---------- emergency keywords ----------
EMERGENCY_KEYWORDS = [
    "chest pain", "breathless", "shortness of breath", "difficulty breathing",
    "bleeding", "vomit blood", "blood in stool", "severe", "collapse",
    "unconscious", "faint", "loss of consciousness", "severe pain", "dangerous",
    "difficulty breathing", "rapid breathing", "painful breathing"
]

def detect_emergency(text):
    if not text:
        return False
    t = str(text).lower()
    for kw in EMERGENCY_KEYWORDS:
        if kw in t:
            return True
    return False

# ---------- LOAD DATA (normalize colnames, tolerant validation) ----------
_lock = threading.Lock()
_cache = {"df": None, "loaded_at": None, "path": None}

def _find_existing_path():
    for p in DATA_PATHS:
        if os.path.exists(p):
            return p
    return None

def _normalize_colname_map(cols):
    """
    Return mapping from lower-collapsed names -> original column name
    """
    return { " ".join(str(c).strip().split()).lower(): c for c in cols }

def load_symptom_df():
    with _lock:
        if _cache["df"] is not None:
            return _cache["df"], _cache["path"]

        found = _find_existing_path()
        if not found:
            raise FileNotFoundError(
                f"No symptom file found. Searched: {DATA_PATHS}. Put symptom_list.csv or symptom_list.xlsx in the app folder or change DATA_PATHS."
            )

        # read file (CSV or Excel)
        try:
            if found.lower().endswith(".csv"):
                df = pd.read_csv(found)
            else:
                # try explicit sheet 'Sheet1' then fallback
                try:
                    df = pd.read_excel(found, sheet_name="Sheet1")
                except Exception:
                    df = pd.read_excel(found)
        except Exception as e:
            raise RuntimeError(f"Failed to read symptom file {found}: {e}")

        # ---- normalize column names: strip & collapse spaces ----
        new_cols = []
        for c in df.columns:
            nc = str(c).strip()
            nc = " ".join(nc.split())
            new_cols.append(nc)
        df.columns = new_cols

        # ---- auto-detect / map alternate names ----
        # Common alternate names mapping => canonical name we will use internally
        preferred_names = {
            "disease": ["disease", "disease name", "name"],
            "typical_symptoms": ["typical symptoms", "typical symptom", "symptoms", "common symptoms"],
            "confirm_symptom": ["confirm symptom", "confirm symptoms", "confirm", "confirmation symptoms"],
            "unique_symptom": ["unique symptom", "unique symptoms"],
            "recommendation": ["recommendation", "recommendations", "advice", "suggestion", "improvement_steps"],
            "severity": ["severity", "seriousness", "urgency"]
        }

        # Build reverse lookup of existing column names
        lowered_map = _normalize_colname_map(df.columns)

        # helper to pick a column if any of candidate substrings present
        def pick_column(candidate_list):
            for cand in candidate_list:
                # exact lowered match first
                if cand in lowered_map:
                    return lowered_map[cand]
            # try substring match
            for lowname, orig in lowered_map.items():
                for cand in candidate_list:
                    if cand in lowname:
                        return orig
            return None

        # select columns if present
        col_selected = {}
        for canon, cand_list in preferred_names.items():
            col_selected[canon] = pick_column(cand_list)

        # Ensure we have at least a disease-like column, else fail
        if not col_selected.get("disease"):
            # try any column that seems like name/disease
            fallback = None
            for low, orig in lowered_map.items():
                if any(k in low for k in ["disease", "name", "condition"]):
                    fallback = orig
                    break
            if fallback:
                col_selected["disease"] = fallback

        if not col_selected.get("disease"):
            raise ValueError(f"No disease-like column found in dataset. Columns: {df.columns.tolist()}")

        # Rename chosen columns to canonical internal names for easier downstream code
        rename_map = {}
        for canon, orig in col_selected.items():
            if orig and orig != canon:
                rename_map[orig] = canon if canon != "typical_symptoms" else "Typical Symptoms"
        # We'll rename a minimal set: disease -> 'Disease' and typical/confirm/unique/recommendation/severity as found.
        # Build a friendly rename dict
        friendly_rename = {}
        if col_selected.get("disease"):
            friendly_rename[col_selected["disease"]] = "Disease"
        if col_selected.get("typical_symptoms"):
            friendly_rename[col_selected["typical_symptoms"]] = "Typical Symptoms"
        if col_selected.get("confirm_symptom"):
            friendly_rename[col_selected["confirm_symptom"]] = "Confirm Symptom"
        if col_selected.get("unique_symptom"):
            friendly_rename[col_selected["unique_symptom"]] = "Unique Symptom"
        if col_selected.get("recommendation"):
            friendly_rename[col_selected["recommendation"]] = "Recommendations"
        if col_selected.get("severity"):
            friendly_rename[col_selected["severity"]] = "Severity"

        if friendly_rename:
            try:
                df = df.rename(columns=friendly_rename)
            except Exception:
                # non-fatal
                pass

        # Ensure expected columns exist (create empty ones if absent) so existing code doesn't crash
        expected_cols = ["Disease", "Typical Symptoms", "Confirm Symptom", "Unique Symptom", "Recommendations", "Severity"]
        for c in expected_cols:
            if c not in df.columns:
                df[c] = ""

        # Basic warning if some commonly-used symptom columns were missing originally
        missing_initial = []
        if "Typical Symptoms" not in friendly_rename.values() and "Typical Symptoms" not in lowered_map.values():
            missing_initial.append("Typical Symptoms (created empty)")
        if "Confirm Symptom" not in friendly_rename.values() and "Confirm Symptom" not in lowered_map.values():
            missing_initial.append("Confirm Symptom (created empty)")
        if "Unique Symptom" not in friendly_rename.values() and "Unique Symptom" not in lowered_map.values():
            missing_initial.append("Unique Symptom (created empty)")

        if missing_initial:
            app.logger.info("Note: some symptom columns were not found and were synthesized empty columns: %s", missing_initial)

        # cache and return
        _cache["df"] = df
        _cache["loaded_at"] = time.time()
        _cache["path"] = found
        return df, found

# Attempt load and surface clear errors (no sys.exit)
try:
    df, data_file_used = load_symptom_df()
    app.logger.info("Loaded symptom data from %s — shape %s", data_file_used, df.shape)
    app.logger.info("Columns found: %s", df.columns.tolist())
except Exception as e:
    # Print full traceback and a helpful message, but DON'T exit the kernel.
    tb = traceback.format_exc()
    print("ERROR loading symptom dataset:\n", tb)
    # To help debugging, expose a lightweight debug endpoint that shows the error
    data_file_used = None
    df = None

    @app.route("/")
    def index_error():
        return "Server started but dataset failed to load. Check notebook output for the traceback."

    @app.route("/_debug_load_error", methods=["GET"])
    def _debug_load_error():
        return jsonify({
            "error": str(e),
            "traceback": tb,
            "searched_paths": DATA_PATHS
        })

# ---------- If df is loaded, normal endpoints; else endpoints still run but return helpful errors ----------
@app.route("/_debug_symptoms", methods=["GET"])
def debug_symptoms():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error."}), 500
    try:
        return jsonify({
            "file": data_file_used,
            "shape": list(df.shape),
            "columns": df.columns.tolist(),
            "sample_first_rows": df.head(5).to_dict(orient="records")
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/predict", methods=["POST"])
def predict():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error."}), 500

    data = request.get_json(force=True, silent=False)
    app.logger.info("Predict request JSON: %s", data)
    tokens = data.get("symptoms", [])

    if isinstance(tokens, str):
        tokens = re.findall(r"\w+", tokens.lower())
    if not isinstance(tokens, (list, tuple)):
        return jsonify({"error": "Invalid payload: 'symptoms' must be list or string"}), 400

    user_text = " ".join(str(t) for t in tokens).lower()
    user_tokens = token_set(user_text)

    possible = []
    for _, row in df.iterrows():
        typical_phrases = split_symptoms(row["Typical Symptoms"])
        phrase_match = any(normalize_text(ph) in normalize_text(user_text) for ph in typical_phrases if ph)
        token_overlap = sum(len(user_tokens & token_set(ph)) for ph in typical_phrases)
        if phrase_match or token_overlap >= 1:
            possible.append({
                "disease": row["Disease"],
                "confirm_symptoms": split_symptoms(row["Confirm Symptom"]),
                "unique_symptom": row.get("Unique Symptom", "")
            })

    app.logger.info("Possible matches found: %s", [p["disease"] for p in possible])
    if not possible:
        return jsonify({"stage": "no_match", "message": "No disease match found."})

    confirm_list = [", ".join(p["confirm_symptoms"]) for p in possible]

    try:
        if openai.api_key:
            question = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a friendly medical assistant."},
                    {"role": "user", "content": f"User reported: {user_text}. Possible diseases: {[p['disease'] for p in possible]}. Confirm symptoms to ask about: {confirm_list}. Ask the user politely about these symptoms in one short question."}
                ],
                max_tokens=80
            )
            ai_question = question["choices"][0]["message"]["content"]
        else:
            raise RuntimeError("OpenAI key not present - using fallback.")
    except Exception as e:
        app.logger.info("OpenAI fallback: %s", e)
        ai_question = f"Please tell me if you have any of these symptoms: {confirm_list}"

    return jsonify({
        "stage": "ask_confirmation",
        "possible_diseases": possible,
        "question": ai_question
    })

@app.route("/confirm", methods=["POST"])
def confirm():
    if df is None:
        return jsonify({"error": "Dataset not loaded. See /_debug_load_error."}), 500

    data = request.get_json(force=True, silent=False)
    user_reply = data.get("reply", "")
    possible = data.get("possible_diseases", [])

    if not isinstance(possible, list):
        return jsonify({"error": "Invalid payload: 'possible_diseases' must be a list"}), 400

    reply_text = str(user_reply).lower()
    best_match = None
    best_count = 0
    for p in possible:
        count = sum(1 for sym in p.get("confirm_symptoms", []) if sym.lower() in reply_text)
        if count > best_count:
            best_count = count
            best_match = p.get("disease")

    if best_match:
        return jsonify({"stage":"final_result","disease":best_match,"confidence":f"{best_count} confirm symptoms matched"})
    else:
        return jsonify({"stage":"final_result","disease":"Unclear","confidence":"No confirm symptoms matched"})

#
# ----------------------- NEW: /recommend endpoint -----------------------
#
# Uses the loaded dataframe to find a matching "Recommendations" entry.
# Returns:
#   {
#     ok: True,
#     matched_disease: "...",
#     advice_text: "full text for UI",
#     advice_html: "<div>...</div>",
#     advice_text_for_speech: "I'm sorry you're not feeling well ...",
#     severity: "...",
#     urgent: False,
#     row_index: 12,
#     source: "excel"
#   }
#
def find_recommendation_row(disease, tokens):
    """
    Returns (idx, row) or (None, None).
    Matching strategy:
      1) exact / normalized disease substring match
      2) token overlap with relevant columns (Typical Symptoms, Confirm Symptom, Recommendations)
      3) fuzzy similarity against disease-like columns
    """
    if df is None or df.empty:
        return None, None

    # find disease-like columns (we renamed earlier to 'Disease' where possible)
    disease_cols = [c for c in df.columns if 'disease' in c.lower() or 'name' in c.lower()] or ["Disease"]

    norm_disease = ""
    if disease:
        norm_disease = re.sub(r'[^a-z0-9 ]', ' ', str(disease).lower()).strip()

    # 1) exact substring disease match (preferred)
    if norm_disease:
        # search disease cols first
        for col in disease_cols:
            for idx, val in df[col].fillna("").iteritems():
                try:
                    if norm_disease and norm_disease in re.sub(r'[^a-z0-9 ]',' ', str(val).lower()):
                        return idx, df.loc[idx]
                except Exception:
                    continue
        # fallback: search whole row text
        for idx, row in df.iterrows():
            combined = " ".join([str(row.get(c,"")) for c in df.columns])
            if norm_disease and norm_disease in re.sub(r'[^a-z0-9 ]',' ', combined.lower()):
                return idx, row

    # 2) token overlap
    token_set_input = set(t.lower() for t in (tokens or []) if t)
    if token_set_input:
        best_idx = None
        best_score = 0
        for idx, row in df.iterrows():
            hay = ""
            # include columns likely to contain symptoms/recommendations
            for c in df.columns:
                if any(k in c.lower() for k in ['typical', 'confirm', 'symptom', 'recommend', 'unique', 'name', 'disease']):
                    hay += " " + str(row.get(c, ""))
            hay_norm = re.sub(r'[^a-z0-9 ]',' ', hay.lower())
            hay_words = set(hay_norm.split())
            score = len(token_set_input & hay_words)
            if score > best_score:
                best_score = score
                best_idx = idx
        if best_idx is not None and best_score > 0:
            return best_idx, df.loc[best_idx]

    # 3) fuzzy similarity against disease-like columns
    if disease_cols and norm_disease:
        best_idx = None
        best_sim = 0.0
        for col in disease_cols:
            for idx, val in df[col].fillna("").iteritems():
                sim = similarity(norm_disease, re.sub(r'[^a-z0-9 ]',' ', str(val).lower()))
                if sim > best_sim:
                    best_sim = sim
                    best_idx = idx
        if best_sim > 0.6:
            return best_idx, df.loc[best_idx]

    return None, None

def short_summary_for_tts(text, max_chars=220):
    if not text:
        return ""
    # take first sentence if short, else truncate gracefully
    parts = re.split(r'(?<=[.!?])\s+', text.strip())
    if parts and parts[0]:
        s = parts[0].strip()
        if len(s) <= max_chars:
            return s
    # fallback: truncate full text
    clean = re.sub(r'\s+', ' ', str(text).strip())
    if len(clean) <= max_chars:
        return clean
    truncated = clean[:max_chars].rsplit(' ', 1)[0]
    return truncated + "..."

@app.route("/recommend", methods=["POST"])
def recommend():
    """
    Expects JSON: { "disease": "Bronchitis ...", "tokens": ["fever","cough"] }
    Returns the recommendation record from the spreadsheet (or a fallback).
    """
    if df is None:
        return jsonify({"ok": False, "message": "Dataset not loaded. See /_debug_load_error."}), 500

    payload = request.get_json(silent=True) or {}
    disease = payload.get("disease", "") or ""
    tokens = payload.get("tokens", []) or []

    try:
        idx, row = find_recommendation_row(disease, tokens)
    except Exception as e:
        app.logger.exception("Error finding recommendation row: %s", e)
        idx, row = None, None

    # attempt to detect recommendations column (we created 'Recommendations' earlier if present)
    rec_col = None
    for c in df.columns:
        if "recommend" in c.lower():
            rec_col = c
            break

    if row is None:
        # fallback generic advice
        generic_text = "I'm sorry you're not feeling well — rest, hydrate, and take paracetamol for fever if needed. If you have severe symptoms such as chest pain or difficulty breathing, seek emergency care."
        return jsonify({
            "ok": True,
            "matched_disease": None,
            "advice_text": generic_text,
            "advice_html": f"<div>{generic_text}</div>",
            "advice_text_for_speech": generic_text,
            "severity": "",
            "urgent": detect_emergency(generic_text),
            "row_index": None,
            "source": "fallback"
        })

    # Compose recommendation text
    rec_text = ""
    if rec_col:
        rec_text = str(row.get(rec_col, "") or "").strip()
    if not rec_text:
        # try to stitch together from other helpful columns
        parts = []
        for c in df.columns:
            if c.lower() not in (rec_col or "").lower():
                try:
                    v = str(row.get(c, "") or "").strip()
                except Exception:
                    v = ""
                if v:
                    parts.append(v)
        rec_text = " ".join(parts).strip()

    if not rec_text:
        rec_text = "No OTC suggestion; seek clinician. Rest and hydrate."

    # Prepare speech-friendly summary
    tts_short = short_summary_for_tts(rec_text, max_chars=220)
    empathetic_speech = f"I'm sorry you're not feeling well — {tts_short}. I can save your phone number for a doctor callback if you want."

    urgent = detect_emergency(rec_text)

    matched_name = None
    # try to get a matched disease name from any disease-like column
    for c in df.columns:
        if 'disease' in c.lower() or 'name' in c.lower():
            try:
                candidate = row.get(c)
                if candidate and str(candidate).strip():
                    matched_name = str(candidate)
                    break
            except Exception:
                continue
    if not matched_name:
        matched_name = disease or ""

    safe_html = f"<div><strong>Advice</strong><div style='margin-top:6px'>{rec_text}</div></div>"

    response = {
        "ok": True,
        "matched_disease": matched_name,
        "advice_text": rec_text,
        "advice_html": safe_html,
        "advice_text_for_speech": empathetic_speech,
        "severity": str(row.get("Severity")) if "Severity" in row.index else "",
        "urgent": bool(urgent),
        "row_index": int(idx) if idx is not None else None,
        "source": "excel"
    }
    return jsonify(response)

# ---------- RUN ----------
if __name__ == "__main__":
    # Use use_reloader=False to avoid child-process reloader in notebooks which causes confusing SystemExit messages
    app.run(host="127.0.0.1", port=5005, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [28/Oct/2025 11:33:29] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:33:29] "POST /confirm HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:42:17] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:42:17] "POST /confirm HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:42:31] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:42:31] "POST /confirm HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:42:46] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:42:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:43:03] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2025 11:43:03] "POST /confirm HTTP/1.1" 200 -


In [ ]:
# main_app_safe.py - COMPLETE FIXED VERSION WITH IMPROVED JSON RECOMMENDATIONS
import os, re, json, traceback, csv
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import openai
from dotenv import load_dotenv
from difflib import SequenceMatcher
import time
import threading
from datetime import datetime

# ---------- CONFIG ----------
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY", "")

DATA_PATHS = [
    "symptom_list_with_recommendations_fixed.csv",
    "symptom_list_with_recommendations.csv",
    "symptom_list.csv",
    "symptom_list.xlsx",
]

RECOMMENDATIONS_JSON = "recommendations.json"
PHONE_LOG_FILE = "phone_callbacks.csv"

# ---------- APP ----------
app = Flask(__name__)
CORS(app)

# ---------- EMERGENCY KEYWORDS ----------
EMERGENCY_KEYWORDS = [
    "chest pain", "breathless", "shortness of breath", "difficulty breathing",
    "bleeding", "vomit blood", "blood in stool", "severe", "collapse",
    "unconscious", "faint", "loss of consciousness", "severe pain", "emergency"
]

# ---------- LOAD RECOMMENDATIONS JSON ----------
recommendations_data = None

def load_recommendations_json():
    """Load recommendations from JSON file with extensive error checking"""
    global recommendations_data
    
    print(f"\n{'='*60}")
    print(f"LOADING RECOMMENDATIONS JSON")
    print(f"{'='*60}")
    print(f"Looking for file: {RECOMMENDATIONS_JSON}")
    print(f"Current directory: {os.getcwd()}")
    print(f"File exists: {os.path.exists(RECOMMENDATIONS_JSON)}")
    
    if not os.path.exists(RECOMMENDATIONS_JSON):
        print(f"⚠️  Warning: {RECOMMENDATIONS_JSON} not found in {os.getcwd()}")
        # Try to find it in parent directories
        for parent in ['../', '../../']:
            alt_path = os.path.join(parent, RECOMMENDATIONS_JSON)
            if os.path.exists(alt_path):
                print(f"Found file at: {alt_path}")
                with open(alt_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    print(f"✓ Loaded {len(data.get('diseases', []))} disease recommendations")
                    return data
        print("⚠️  Using fallback recommendations.")
        return None
    
    try:
        with open(RECOMMENDATIONS_JSON, 'r', encoding='utf-8') as f:
            data = json.load(f)
            diseases = data.get('diseases', [])
            print(f"✓ Successfully loaded JSON")
            print(f"  Number of diseases: {len(diseases)}")
            if diseases:
                print(f"  First disease: {diseases[0].get('name', 'N/A')}")
                print(f"  Sample aliases: {diseases[0].get('aliases', [])}")
            print(f"{'='*60}\n")
            return data
    except json.JSONDecodeError as e:
        print(f"✗ JSON parsing error: {e}")
        print(f"  Line {e.lineno}, Column {e.colno}")
        return None
    except Exception as e:
        print(f"✗ Error loading recommendations JSON: {e}")
        traceback.print_exc()
        return None

recommendations_data = load_recommendations_json()

# ---------- UTIL ----------
def normalize_text(s):
    """Aggressive normalization for matching"""
    if not s:
        return ""
    # Convert to lowercase, remove special chars, normalize spaces
    text = str(s).lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # Replace punctuation with space
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
    return text.strip()

def token_set(s):
    return set(t for t in normalize_text(s).split() if t and len(t) > 1)

def split_symptoms(sym_str):
    return [s.strip() for s in str(sym_str).split(",") if s.strip()]

def similarity(a, b):
    """Calculate similarity between two strings"""
    return SequenceMatcher(None, normalize_text(a), normalize_text(b)).ratio()

def token_overlap_score(a, b):
    """Calculate token overlap score"""
    tokens_a = token_set(a)
    tokens_b = token_set(b)
    if not tokens_a or not tokens_b:
        return 0.0
    intersection = len(tokens_a & tokens_b)
    union = len(tokens_a | tokens_b)
    return intersection / union if union > 0 else 0.0

def detect_emergency(text):
    """Detect emergency keywords"""
    if not text:
        return False
    t = str(text).lower()
    for kw in EMERGENCY_KEYWORDS:
        if kw in t:
            return True
    return False

# ---------- LOAD CSV DATA ----------
_lock = threading.Lock()
_cache = {"df": None, "loaded_at": None, "path": None}

def _find_existing_path():
    for p in DATA_PATHS:
        if os.path.exists(p):
            return p
    return None

def _normalize_colname_map(cols):
    return { " ".join(str(c).strip().split()).lower(): c for c in cols }

def load_symptom_df():
    with _lock:
        if _cache["df"] is not None:
            return _cache["df"], _cache["path"]

        found = _find_existing_path()
        if not found:
            raise FileNotFoundError(f"No symptom file found. Searched: {DATA_PATHS}")

        try:
            if found.lower().endswith(".csv"):
                encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
                df = None
                for encoding in encodings_to_try:
                    try:
                        df = pd.read_csv(found, encoding=encoding)
                        print(f"✓ Loaded CSV with encoding: {encoding}")
                        break
                    except UnicodeDecodeError:
                        continue
                if df is None:
                    raise RuntimeError("Failed to read CSV with any encoding")
            else:
                df = pd.read_excel(found)
        except Exception as e:
            raise RuntimeError(f"Failed to read symptom file {found}: {e}")

        # Normalize columns
        new_cols = [" ".join(str(c).strip().split()) for c in df.columns]
        df.columns = new_cols

        # Auto-detect and rename columns
        preferred_names = {
            "disease": ["disease", "disease name", "name"],
            "typical_symptoms": ["typical symptoms", "typical symptom", "symptoms"],
            "confirm_symptom": ["confirm symptom", "confirm symptoms", "confirm"],
            "unique_symptom": ["unique symptom", "unique symptoms"],
        }

        lowered_map = _normalize_colname_map(df.columns)

        def pick_column(candidate_list):
            for cand in candidate_list:
                if cand in lowered_map:
                    return lowered_map[cand]
            return None

        col_selected = {}
        for canon, cand_list in preferred_names.items():
            col_selected[canon] = pick_column(cand_list)

        friendly_rename = {}
        if col_selected.get("disease"):
            friendly_rename[col_selected["disease"]] = "Disease"
        if col_selected.get("typical_symptoms"):
            friendly_rename[col_selected["typical_symptoms"]] = "Typical Symptoms"
        if col_selected.get("confirm_symptom"):
            friendly_rename[col_selected["confirm_symptom"]] = "Confirm Symptom"
        if col_selected.get("unique_symptom"):
            friendly_rename[col_selected["unique_symptom"]] = "Unique Symptom"

        if friendly_rename:
            df = df.rename(columns=friendly_rename)

        for c in ["Disease", "Typical Symptoms", "Confirm Symptom", "Unique Symptom"]:
            if c not in df.columns:
                df[c] = ""

        _cache["df"] = df
        _cache["loaded_at"] = time.time()
        _cache["path"] = found
        
        print(f"✓ CSV loaded successfully: {df.shape[0]} diseases")
        return df, found

try:
    df, data_file_used = load_symptom_df()
    app.logger.info("Loaded symptom data from %s — shape %s", data_file_used, df.shape)
except Exception as e:
    tb = traceback.format_exc()
    print("ERROR loading symptom dataset:\n", tb)
    df = None

# ---------- JSON-BASED RECOMMENDATION FINDER (IMPROVED) ----------

def find_recommendation_json(disease_name):
    """
    IMPROVED: Find recommendation with better matching algorithm
    """
    if not recommendations_data or 'diseases' not in recommendations_data:
        print("⚠️  No recommendations data available")
        return None
    
    disease_norm = normalize_text(disease_name)
    disease_tokens = token_set(disease_name)
    
    print(f"\n{'='*60}")
    print(f"SEARCHING FOR RECOMMENDATION")
    print(f"{'='*60}")
    print(f"Original: '{disease_name}'")
    print(f"Normalized: '{disease_norm}'")
    print(f"Tokens: {disease_tokens}")
    
    best_match = None
    best_score = 0.0
    best_method = None
    
    for idx, disease_entry in enumerate(recommendations_data['diseases']):
        entry_name = disease_entry.get('name', '')
        entry_name_norm = normalize_text(entry_name)
        
        if idx < 3:  # Show first few checks
            print(f"\n[{idx+1}] Checking: {entry_name}")
            print(f"    Normalized: {entry_name_norm}")
        
        # METHOD 1: EXACT MATCH (normalized)
        if disease_norm == entry_name_norm:
            print(f"  ✅✅✅ EXACT MATCH!")
            return disease_entry
        
        # METHOD 2: EXACT SUBSTRING MATCH
        if disease_norm in entry_name_norm or entry_name_norm in disease_norm:
            print(f"  ✅✅ SUBSTRING MATCH!")
            return disease_entry
        
        # METHOD 3: Token overlap
        entry_tokens = token_set(entry_name)
        overlap = len(disease_tokens & entry_tokens)
        overlap_score = overlap / max(len(disease_tokens), len(entry_tokens)) if disease_tokens or entry_tokens else 0
        
        if overlap_score > 0.6 and overlap >= 2:
            if idx < 3:
                print(f"  ✅ TOKEN OVERLAP MATCH (score: {overlap_score:.2f}, overlap: {overlap})")
            if overlap_score > best_score:
                best_score = overlap_score
                best_match = disease_entry
                best_method = "token_overlap"
        
        # METHOD 4: Sequence similarity
        sim_score = similarity(disease_name, entry_name)
        if sim_score > best_score:
            best_score = sim_score
            best_match = disease_entry
            best_method = "similarity"
            if idx < 3:
                print(f"  ~ Better similarity: {sim_score:.2f}")
        
        # METHOD 5: Check aliases
        for alias in disease_entry.get('aliases', []):
            alias_norm = normalize_text(alias)
            
            # Exact alias match
            if disease_norm == alias_norm:
                print(f"  ✅✅✅ EXACT ALIAS MATCH: '{alias}'")
                return disease_entry
            
            # Substring in alias
            if disease_norm in alias_norm or alias_norm in disease_norm:
                print(f"  ✅✅ ALIAS SUBSTRING MATCH: '{alias}'")
                return disease_entry
            
            # Alias similarity
            alias_sim = similarity(disease_name, alias)
            if alias_sim > best_score:
                best_score = alias_sim
                best_match = disease_entry
                best_method = f"alias_similarity:{alias}"
    
    # Return best match if good enough
    if best_match and best_score >= 0.50:  # Lowered threshold
        print(f"\n✅ FUZZY MATCH: {best_match.get('name')}")
        print(f"  Method: {best_method}")
        print(f"  Score: {best_score:.2f}")
        return best_match
    
    print(f"\n❌ NO MATCH FOUND")
    print(f"  Best score: {best_score:.2f}")
    print(f"  Best candidate: {best_match.get('name') if best_match else 'None'}")
    return None

def format_recommendation_html_from_json(recommendation):
    """Format JSON recommendation into HTML"""
    if not recommendation:
        return "<div class='small-muted'>No recommendations available.</div>"
    
    medications = recommendation.get('medications', '').strip()
    home_remedies = recommendation.get('home_remedies', '').strip()
    
    html_parts = []
    
    # Check for urgent conditions
    urgent_keywords = ['seek clinician', 'consult a doctor', 'please consult', 'medical attention', 'no otc']
    is_urgent = any(kw in medications.lower() for kw in urgent_keywords)
    
    if medications:
        if is_urgent:
            html_parts.append(
                f"<div class='urgent-warning'>"
                f"<strong>⚠️ Medical Attention Required</strong>"
                f"<div style='margin-top:8px;color:#475569;'>{medications}</div>"
                f"</div>"
            )
        elif medications.lower() != 'please consult a doctor':
            html_parts.append(
                f"<div class='medication'>"
                f"<strong>💊 Medications</strong>"
                f"<div style='margin-top:6px;'>{medications}</div>"
                f"</div>"
            )
    
    if home_remedies and home_remedies.lower() not in ['rest and stay hydrated', 'rest and stay hydrated.']:
        html_parts.append(
            f"<div class='home-remedy'>"
            f"<strong>🏡 Home Remedies</strong>"
            f"<div style='margin-top:6px;'>{home_remedies}</div>"
            f"</div>"
        )
    
    if not html_parts:
        html_parts.append(
            "<div class='small-muted' style='padding:10px;'>"
            "Please rest, stay hydrated, and consult a healthcare professional if symptoms persist."
            "</div>"
        )
    
    return "".join(html_parts)

# ---------- ENDPOINTS ----------

@app.route("/_debug_symptoms", methods=["GET"])
def debug_symptoms():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500
    return jsonify({
        "file": data_file_used,
        "shape": list(df.shape),
        "columns": df.columns.tolist(),
        "sample_first_rows": df.head(5).to_dict(orient="records"),
        "recommendations_loaded": recommendations_data is not None,
        "num_recommendations": len(recommendations_data.get('diseases', [])) if recommendations_data else 0
    })

@app.route("/_test_json_load", methods=["GET"])
def test_json_load():
    """Test if JSON is properly loaded"""
    return jsonify({
        "json_file_path": RECOMMENDATIONS_JSON,
        "file_exists": os.path.exists(RECOMMENDATIONS_JSON),
        "json_loaded": recommendations_data is not None,
        "num_diseases": len(recommendations_data.get('diseases', [])) if recommendations_data else 0,
        "first_5_diseases": [
            d.get('name') for d in recommendations_data.get('diseases', [])[:5]
        ] if recommendations_data else [],
        "current_directory": os.getcwd(),
        "directory_contents": os.listdir('.')[:20]
    })

@app.route("/_test_match/<path:disease_name>", methods=["GET"])
def test_match(disease_name):
    """Test matching for a specific disease name"""
    print(f"\n{'='*60}")
    print(f"MANUAL TEST FOR: {disease_name}")
    print(f"{'='*60}")
    
    result = find_recommendation_json(disease_name)
    
    if result:
        return jsonify({
            "success": True,
            "searched_for": disease_name,
            "matched_disease": result.get('name'),
            "medications": result.get('medications'),
            "home_remedies": result.get('home_remedies'),
            "full_data": result
        })
    else:
        # Try to find close matches
        close_matches = []
        if recommendations_data and 'diseases' in recommendations_data:
            for d in recommendations_data['diseases'][:20]:
                score = similarity(disease_name, d.get('name', ''))
                if score > 0.3:
                    close_matches.append({
                        "name": d.get('name'),
                        "score": round(score, 3)
                    })
        
        return jsonify({
            "success": False,
            "searched_for": disease_name,
            "normalized": normalize_text(disease_name),
            "close_matches": sorted(close_matches, key=lambda x: x['score'], reverse=True)[:5]
        })

@app.route("/_list_all_diseases", methods=["GET"])
def list_all_diseases():
    """List all diseases in JSON for reference"""
    if not recommendations_data or 'diseases' not in recommendations_data:
        return jsonify({"error": "No data loaded"}), 500
    
    diseases = []
    for d in recommendations_data['diseases']:
        diseases.append({
            "name": d.get('name'),
            "normalized": normalize_text(d.get('name', '')),
            "aliases": d.get('aliases', []),
            "has_medications": bool(d.get('medications')),
            "has_home_remedies": bool(d.get('home_remedies'))
        })
    
    return jsonify({
        "total": len(diseases),
        "diseases": diseases
    })

@app.route("/predict", methods=["POST"])
def predict():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500

    data = request.get_json(force=True)
    tokens = data.get("symptoms", [])

    if isinstance(tokens, str):
        tokens = re.findall(r"\w+", tokens.lower())

    user_text = " ".join(str(t) for t in tokens).lower()
    user_tokens = token_set(user_text)

    possible = []
    for _, row in df.iterrows():
        typical_phrases = split_symptoms(row["Typical Symptoms"])
        phrase_match = any(normalize_text(ph) in normalize_text(user_text) for ph in typical_phrases if ph)
        token_overlap = sum(len(user_tokens & token_set(ph)) for ph in typical_phrases)
        if phrase_match or token_overlap >= 1:
            possible.append({
                "disease": row["Disease"],
                "confirm_symptoms": split_symptoms(row["Confirm Symptom"]),
                "unique_symptom": row.get("Unique Symptom", "")
            })

    if not possible:
        return jsonify({"stage": "no_match", "message": "No disease match found."})

    confirm_list = [", ".join(p["confirm_symptoms"]) for p in possible]

    try:
        if openai.api_key:
            question = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a friendly medical assistant."},
                    {"role": "user", "content": f"User: {user_text}. Possible: {[p['disease'] for p in possible]}. Ask about: {confirm_list}"}
                ],
                max_tokens=80
            )
            ai_question = question["choices"][0]["message"]["content"]
        else:
            raise RuntimeError("No OpenAI key")
    except Exception:
        ai_question = f"Please tell me if you have any of these symptoms: {confirm_list}"

    return jsonify({
        "stage": "ask_confirmation",
        "possible_diseases": possible,
        "question": ai_question
    })

@app.route("/confirm", methods=["POST"])
def confirm():
    if df is None:
        return jsonify({"error": "Dataset not loaded"}), 500

    data = request.get_json(force=True)
    user_reply = data.get("reply", "")
    possible = data.get("possible_diseases", [])

    reply_text = str(user_reply).lower()
    reply_tokens = set(reply_text.split())
    
    explicit_yes = any(word in reply_tokens for word in ['yes', 'yeah', 'yep', 'correct', 'right'])
    explicit_no = any(word in reply_tokens for word in ['no', 'nope', 'not', 'none'])
    
    best_match = None
    best_disease_name = None
    best_count = 0
    
    for p in possible:
        disease_name = p.get("disease", "Unknown")
        confirm_symptoms = p.get("confirm_symptoms", [])
        
        count = sum(1 for sym in confirm_symptoms if str(sym).lower().replace("-", " ") in reply_text)
        
        if count > best_count:
            best_count = count
            best_match = disease_name
            best_disease_name = disease_name

    if explicit_yes and not explicit_no and possible:
        best_disease_name = possible[0].get("disease", "Unknown")
        return jsonify({
            "stage": "final_result",
            "disease": best_disease_name,
            "confidence": 0.85
        })
    elif explicit_no:
        return jsonify({
            "stage": "final_result",
            "disease": "Symptoms unclear",
            "confidence": 0.0
        })
    elif best_match and best_count > 0:
        return jsonify({
            "stage": "final_result",
            "disease": best_disease_name,
            "confidence": min(0.9, 0.5 + (best_count * 0.1))
        })
    else:
        fallback = possible[0].get("disease", "Unknown") if possible else "Unclear"
        return jsonify({
            "stage": "final_result",
            "disease": fallback,
            "confidence": 0.3
        })

@app.route("/recommend", methods=["POST"])
def recommend():
    """JSON-based recommendation endpoint with improved matching"""
    
    print("\n" + "="*80)
    print("RECOMMEND ENDPOINT CALLED")
    print("="*80)
    
    payload = request.get_json(silent=True) or {}
    disease = payload.get("disease", "").strip()
    tokens = payload.get("tokens", [])

    print(f"Raw payload: {payload}")
    print(f"Requested disease: '{disease}'")
    print(f"Tokens: {tokens[:10] if tokens else 'none'}")

    if not disease:
        print("❌ No disease name provided")
        return jsonify({
            "ok": False,
            "advice_text": "No disease specified",
            "advice_html": "<div class='small-muted'>Unable to provide recommendations.</div>",
            "urgent": False
        })

    # Check if JSON is loaded
    if not recommendations_data or 'diseases' not in recommendations_data:
        print("❌ Recommendations data not loaded!")
        return jsonify({
            "ok": False,
            "advice_text": "Recommendation database not available",
            "advice_html": "<div class='small-muted'>Please rest, stay hydrated, and consult a doctor.</div>",
            "urgent": False
        })

    print(f"Searching through {len(recommendations_data['diseases'])} diseases...")

    # Find matching recommendation
    recommendation = find_recommendation_json(disease)
    
    if recommendation:
        print(f"✅ MATCH FOUND: {recommendation.get('name')}")
        
        empathetic_msg = recommendation.get('empathetic_message', '')
        medications = recommendation.get('medications', '')
        home_remedies = recommendation.get('home_remedies', '')
        
        # Create HTML
        html = format_recommendation_html_from_json(recommendation)
        
        # Build speech text
        speech_parts = []
        if medications and 'consult' not in medications.lower():
            speech_parts.append(f"You can take {medications}")
        if home_remedies and home_remedies.lower() not in ['rest and stay hydrated', 'rest and stay hydrated.']:
            speech_parts.append(f"Try these home remedies: {home_remedies}")
        
        speech_text = ". ".join(speech_parts) if speech_parts else "Please rest, stay hydrated, and consult a doctor if symptoms worsen."
        
        urgent = detect_emergency(medications + ' ' + home_remedies) or 'seek' in medications.lower() or 'consult' in medications.lower()
        
        print(f"✓ Returning recommendation")
        print(f"  Medications: {medications[:50]}...")
        print(f"  Home remedies: {home_remedies[:50]}...")
        print(f"  Urgent: {urgent}")
        
        return jsonify({
            "ok": True,
            "matched_disease": recommendation.get('name'),
            "advice_text": f"Medications: {medications}; Home remedies: {home_remedies}",
            "advice_html": html,
            "advice_text_for_speech": speech_text,
            "full_message": empathetic_msg,
            "urgent": urgent,
            "source": "json_database"
        })
    else:
        # Fallback
        print("❌ NO MATCH FOUND - Using fallback")
        print(f"  Searched for: '{disease}'")
        print(f"  Normalized: '{normalize_text(disease)}'")
        
        generic = "I'm sorry, I couldn't find specific recommendations for this condition. Please rest, stay hydrated, and consult a doctor if symptoms worsen."
        return jsonify({
            "ok": True,
            "matched_disease": None,
            "advice_text": generic,
            "advice_html": f"<div class='small-muted'>{generic}</div>",
            "advice_text_for_speech": generic,
            "urgent": False,
            "source": "fallback"
        })

@app.route("/save_phone", methods=["POST"])
def save_phone():
    try:
        payload = request.get_json(silent=True) or {}
        phone = payload.get("phone", "").strip()
        disease = payload.get("disease", "Unknown")
        confidence = payload.get("confidence", 0)
        
        if not phone:
            return jsonify({"ok": False, "message": "Phone required"}), 400
        
        digits = re.sub(r'[^\d]', '', phone)
        if not (10 <= len(digits) <= 14):
            return jsonify({"ok": False, "message": "Invalid phone format"}), 400
        
        file_exists = os.path.exists(PHONE_LOG_FILE)
        with open(PHONE_LOG_FILE, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(['Timestamp', 'Phone', 'Disease', 'Confidence', 'Status'])
            writer.writerow([datetime.now().strftime('%Y-%m-%d %H:%M:%S'), phone, disease, confidence, 'Pending'])
        
        return jsonify({
            "ok": True,
            "message": "Phone saved successfully",
            "phone": phone
        })
    except Exception as e:
        return jsonify({"ok": False, "message": str(e)}), 500

if __name__ == "__main__":
    print("\n" + "="*60)
    print("STARTUP CHECK")
    print("="*60)
    print(f"CSV loaded: {df is not None}")
    if df is not None:
        print(f"CSV shape: {df.shape}")
        print(f"CSV columns: {df.columns.tolist()}")
    print(f"Recommendations JSON loaded: {recommendations_data is not None}")
    if recommendations_data:
        print(f"Number of diseases in JSON: {len(recommendations_data.get('diseases', []))}")
        print(f"First 3 diseases: {[d.get('name') for d in recommendations_data.get('diseases', [])[:3]]}")
    print("="*60 + "\n")
    
    app.run(host="127.0.0.1", port=5005, debug=True, use_reloader=False)


LOADING RECOMMENDATIONS JSON
Looking for file: recommendations.json
Current directory: c:\Users\Jay\Desktop\SE proj\New folder\Back
File exists: True
✓ Successfully loaded JSON
  Number of diseases: 62
  First disease: Common Cold (Upper Respiratory Tract Infection)
  Sample aliases: ['common cold (upper respiratory tract infection)']

✓ Loaded CSV with encoding: utf-8
✓ CSV loaded successfully: 62 diseases

STARTUP CHECK
CSV loaded: True
CSV shape: (62, 6)
CSV columns: ['Sr No.', 'Disease', 'Typical Symptoms', 'Unique Symptom', 'Confirm Symptom', 'Recommendations']
Recommendations JSON loaded: True
Number of diseases in JSON: 62
First 3 diseases: ['Common Cold (Upper Respiratory Tract Infection)', 'Influenza (Flu)', 'COVID-19']

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5005
Press CTRL+C to quit
127.0.0.1 - - [29/Oct/2025 03:34:38] "GET /_test_json_load HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:34:44] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:34:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:34:49] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:34:49] "POST /confirm HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:34:49] "OPTIONS /recommend HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:34:49] "POST /recommend HTTP/1.1" 200 -



RECOMMEND ENDPOINT CALLED
Raw payload: {'disease': 'Common Cold (Upper Respiratory Tract Infection)', 'tokens': ['I', 'have', 'fever', 'confirmation', 'sore', 'throat', 'and', 'congestion']}
Requested disease: 'Common Cold (Upper Respiratory Tract Infection)'
Tokens: ['I', 'have', 'fever', 'confirmation', 'sore', 'throat', 'and', 'congestion']
Searching through 62 diseases...

SEARCHING FOR RECOMMENDATION
Original: 'Common Cold (Upper Respiratory Tract Infection)'
Normalized: 'common cold upper respiratory tract infection'
Tokens: {'common', 'cold', 'respiratory', 'tract', 'infection', 'upper'}

[1] Checking: Common Cold (Upper Respiratory Tract Infection)
    Normalized: common cold upper respiratory tract infection
  ✅✅✅ EXACT MATCH!
✅ MATCH FOUND: Common Cold (Upper Respiratory Tract Infection)
✓ Returning recommendation
  Medications: paracetamol, cetirizine...
  Home remedies: rest, warm saline gargle, steam inhalation...
  Urgent: False


127.0.0.1 - - [29/Oct/2025 03:39:22] "OPTIONS /confirm HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:39:22] "POST /confirm HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:39:22] "OPTIONS /recommend HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 03:39:22] "POST /recommend HTTP/1.1" 200 -



RECOMMEND ENDPOINT CALLED
Raw payload: {'disease': 'Common Cold (Upper Respiratory Tract Infection)', 'tokens': ['I', 'have', 'a', 'fever', 'and', 'cough', 'confirmation', 'sore', 'throat', 'and', 'cough']}
Requested disease: 'Common Cold (Upper Respiratory Tract Infection)'
Tokens: ['I', 'have', 'a', 'fever', 'and', 'cough', 'confirmation', 'sore', 'throat', 'and']
Searching through 62 diseases...

SEARCHING FOR RECOMMENDATION
Original: 'Common Cold (Upper Respiratory Tract Infection)'
Normalized: 'common cold upper respiratory tract infection'
Tokens: {'common', 'cold', 'respiratory', 'tract', 'infection', 'upper'}

[1] Checking: Common Cold (Upper Respiratory Tract Infection)
    Normalized: common cold upper respiratory tract infection
  ✅✅✅ EXACT MATCH!
✅ MATCH FOUND: Common Cold (Upper Respiratory Tract Infection)
✓ Returning recommendation
  Medications: paracetamol, cetirizine...
  Home remedies: rest, warm saline gargle, steam inhalation...
  Urgent: False


...

127.0.0.1 - - [29/Oct/2025 09:38:50] "OPTIONS /save_phone HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 09:38:50] "POST /save_phone HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 09:38:59] "GET /_test_json_load HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 09:39:00] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:02] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:02] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:02] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:02] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:03] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:03] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:03] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:03] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:03] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:39:31] "GET /get_patients HTTP


RECOMMEND ENDPOINT CALLED
Raw payload: {'disease': 'Common Cold (Upper Respiratory Tract Infection)', 'tokens': ['I', 'have', 'a', 'fever', 'and', 'cough', 'confirmation', 'congestion', 'and', 'sore', 'throat']}
Requested disease: 'Common Cold (Upper Respiratory Tract Infection)'
Tokens: ['I', 'have', 'a', 'fever', 'and', 'cough', 'confirmation', 'congestion', 'and', 'sore']
Searching through 62 diseases...

SEARCHING FOR RECOMMENDATION
Original: 'Common Cold (Upper Respiratory Tract Infection)'
Normalized: 'common cold upper respiratory tract infection'
Tokens: {'common', 'cold', 'respiratory', 'tract', 'infection', 'upper'}

[1] Checking: Common Cold (Upper Respiratory Tract Infection)
    Normalized: common cold upper respiratory tract infection
  ✅✅✅ EXACT MATCH!
✅ MATCH FOUND: Common Cold (Upper Respiratory Tract Infection)
✓ Returning recommendation
  Medications: paracetamol, cetirizine...
  Home remedies: rest, warm saline gargle, steam inhalation...
  Urgent: False


127.0.0.1 - - [29/Oct/2025 09:43:49] "OPTIONS /save_phone HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 09:43:49] "POST /save_phone HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 09:43:59] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:43:59] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:00] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:01] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:01] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:01] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:01] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:01] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:01] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:02] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:29] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:44:59] "GET /get_patients HTTP/1.


RECOMMEND ENDPOINT CALLED
Raw payload: {'disease': 'Common Cold (Upper Respiratory Tract Infection)', 'tokens': ['I', 'have', 'a', 'fever', 'confirmation', 'sore', 'throat', 'and', 'congestion']}
Requested disease: 'Common Cold (Upper Respiratory Tract Infection)'
Tokens: ['I', 'have', 'a', 'fever', 'confirmation', 'sore', 'throat', 'and', 'congestion']
Searching through 62 diseases...

SEARCHING FOR RECOMMENDATION
Original: 'Common Cold (Upper Respiratory Tract Infection)'
Normalized: 'common cold upper respiratory tract infection'
Tokens: {'common', 'cold', 'respiratory', 'tract', 'infection', 'upper'}

[1] Checking: Common Cold (Upper Respiratory Tract Infection)
    Normalized: common cold upper respiratory tract infection
  ✅✅✅ EXACT MATCH!
✅ MATCH FOUND: Common Cold (Upper Respiratory Tract Infection)
✓ Returning recommendation
  Medications: paracetamol, cetirizine...
  Home remedies: rest, warm saline gargle, steam inhalation...
  Urgent: False


127.0.0.1 - - [29/Oct/2025 09:54:57] "OPTIONS /save_phone HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 09:54:57] "POST /save_phone HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 09:55:03] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:03] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:05] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:05] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:05] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:05] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:05] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:06] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:55:34] "GET /get_patients HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2025 09:56:04] "GET /get_patients HTTP/1.1" 404 -
